In [1]:
import scipy.io
import torch
import numpy as np
import torch.nn as nn
import torch.utils.data as Data
import matplotlib.pyplot as plt
import torch.nn.functional as F
import math
#from tensorboardX import SummaryWriter
from sklearn.metrics import roc_auc_score,roc_curve,auc,average_precision_score,precision_recall_curve
torch.manual_seed(1337)
np.random.seed(1337)
torch.cuda.manual_seed(1337)
torch.backends.cudnn.benchmark=True

In [2]:
print('starting loading the data')
np_test_data = scipy.io.loadmat('test.mat')

starting loading the data


In [3]:
y_test_Pol2 = np_test_data['testdata'][:, [332,354,408,431,432,433,451,453,455,457,459,461,463]].sum(axis=1) > 0

X_test_NRSF = np_test_data['testxdata'][y_test_Pol2]

In [4]:
print('compling the network')
class DanQ(nn.Module):
    def __init__(self, ):
        super(DanQ, self).__init__()
        self.Conv1 = nn.Conv1d(in_channels=4, out_channels=320, kernel_size=13)
        #nn.init.uniform_(self.Conv1.weight, -0.05, 0.05)
        self.Maxpool = nn.MaxPool1d(kernel_size=13, stride=6)
        self.Drop1 = nn.Dropout(p=0.2)
        self.BiLSTM = nn.LSTM(input_size=320, hidden_size=320, num_layers=2,
                                 batch_first=True,
                                 dropout=0.5,
                                 bidirectional=True)
        self.Linear1 = nn.Linear(163*640, 925)
        self.Linear2 = nn.Linear(925, 919)


    def forward(self, input):
        x = self.Conv1(input)
        x1 = F.relu(x)
        x = self.Maxpool(x1)
        x = self.Drop1(x)
        x_x = torch.transpose(x, 1, 2)
        x, (h_n,h_c) = self.BiLSTM(x_x)
        #x, h_n = self.BiGRU(x_x)
        x = x.contiguous().view(-1, 163*640)
        x = self.Linear1(x)
        x = F.relu(x)
        x = self.Linear2(x)
        #x = torch.sigmoid(x)
        return x1,x

danq = DanQ()
danq.load_state_dict(torch.load('model/model0512_2/danq_net_params_4.pkl'))

compling the network


In [5]:
motifs = np.zeros((320, 4, 13))
nsites = np.zeros(320)
danq.eval()
for i in range(0, len(X_test_NRSF), 100):
    x = X_test_NRSF[i:i+100]
    x_tensor = torch.FloatTensor(x)
    #print(seq.shape)
    conv_output, _ = danq(x_tensor)
    max_inds = np.argmax(conv_output.cpu().detach().numpy().data, axis=2)
    max_acts = np.max(conv_output.cpu().detach().numpy().data, axis=2)
    #print(max_inds.shape)
    #print(max_acts.shape)
    for m in range(320):
        for n in range(len(x)):
            if max_acts[n, m] > 0:
                nsites[m] += 1
                motifs[m] += x[n, :, max_inds[n, m]:max_inds[n, m]+13]

In [6]:
motifs_trans = np.transpose(motifs,(0, 2, 1))
print(motifs_trans.shape)
print('Making motifs')

motifs = motifs_trans[:, :, [0, 2, 1, 3]]

for st in range(0,4):
    motifs_file = open('motifs_Pol2_heatmap_%s.txt' % str(st), 'w')
    for m in range(320):
        if nsites[m] == 0:
            continue
        for j in range(13):
            p = 1.0 * motifs[m, j, 0:4] / np.sum(motifs[m, j, 0:4])
            if p[st]==0:
                entropy = 0
            else:
                entropy = (-p[st])*math.log(p[st],2)
            motifs_file.write("%.5f\t" % entropy)
        motifs_file.write('\n')

    motifs_file.close()

(320, 13, 4)
Making motifs


In [7]:
motifs_file = open('motifs_Pol2_heatmap_sum.txt', 'w')
for m in range(320):
    if nsites[m] == 0:
        continue
    for j in range(13):
        p = 1.0 * motifs[m, j, 0:4] / np.sum(motifs[m, j, 0:4])
        for st in range(0,4):
            if p[st]==0 or p[st]==1:
                continue
            else:
                entropy += (-p[st])*math.log(p[st],2)
        motifs_file.write("%.5f\t" % entropy)
        entropy = 0  
    motifs_file.write('\n')

motifs_file.close()

In [23]:
i = 0.200132
(-i)*math.log(i,2)

0.4645016149106253